<a href="https://colab.research.google.com/github/matthold86/DeepLearningWPython/blob/main/predicting_housing_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Housing Prices

________________________________________________

#### **Summary**

This notebook will walk through an example of linear regression to predict the median home price in a Boston suburb, given parameters such as the crime rate, population density. The parameters can be continuous, ordinal, or binary. The dataset contains 550 observations.

________________________________________________

#### **Step 0: Install Packages**

#### **Step 1: Establish Google Drive Connection**
This first step mounts the Colab notebook to the `/DeepLearningWPython` directory on my Google Drive. Any data required for this project (data not accessible via link or python package) will be stored in `/DeepLearningWPython` for its convenient integration with Google Colab.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Github/DeepLearningWPython/

/content/drive/MyDrive/Github/DeepLearningWPython


#### **Step 2: Prepare the Data**

In [4]:
from keras.datasets import boston_housing
from keras import models, layers
import numpy as np

In [5]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


In [6]:
# Normalize the values for each parameter around zero
mean_parameters = train_data.mean(axis=0)
std_parameters = train_data.std(axis=0)
train_data = (train_data - mean_parameters)/std_parameters
test_data = (test_data - mean_parameters)/std_parameters

In [12]:
train_data

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

In [7]:
# Create function for generating the model

# No activation function is requred for final layer because this model will use multiple linear regression to predict a scalar output.
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))

# Loss function: 'MSE' (Mean Squared Error)
# Metrics: 'MAE' (Mean Absolute Error) Units: $Thousands
  model.compile(
      loss='mse',
      optimizer='rmsprop',
      metrics=['mae']
  )
  return model

In [11]:
# Prepare data for K-Fold Cross Validation
folds = 4
section_size = train_data.shape[0] // folds
validation_accuracy_scores = []
epochs = 20
batch_size = 1

for i in range(folds):
  val_data = train_data[i*section_size:(i+1)*section_size]
  val_targets = train_data[i*section_size:(i+1)*section_size]
  training_data = np.concatenate(
      (train_data[:i*section_size],
      train_data[(i+1)*section_size:]),
      axis = 0
  )
  training_targets = np.concatenate(
      (train_targets[:i*section_size],
      train_targets[(i+1)*section_size:]),
      axis = 0
  )
  model = create_model()
  history = model.fit(
      training_data,
      training_targets,
      epochs = epochs,
      batch_size = batch_size,
  )

  val_mse, val_mae  = model.evaluate(val_data, val_targets, verbose=0)
  validation_accuracy_scores.append(val_mae)

print(validation_accuracy_scores)




Epoch 1/20
303/303 [==============================] - 2s 3ms/step - loss: 188.2908 - mae: 10.3106
Epoch 2/20
303/303 [==============================] - 1s 2ms/step - loss: 26.2229 - mae: 3.6350
Epoch 3/20
303/303 [==============================] - 1s 3ms/step - loss: 19.6872 - mae: 3.0481
Epoch 4/20
303/303 [==============================] - 1s 3ms/step - loss: 17.1510 - mae: 2.7886
Epoch 5/20
303/303 [==============================] - 1s 3ms/step - loss: 14.7104 - mae: 2.5944
Epoch 6/20
303/303 [==============================] - 1s 3ms/step - loss: 13.7805 - mae: 2.5544
Epoch 7/20
303/303 [==============================] - 1s 3ms/step - loss: 13.1105 - mae: 2.4434
Epoch 8/20
303/303 [==============================] - 1s 3ms/step - loss: 12.5410 - mae: 2.4093
Epoch 9/20
303/303 [==============================] - 1s 2ms/step - loss: 12.0151 - mae: 2.3649
Epoch 10/20
303/303 [==============================] - 1s 2ms/step - loss: 11.5748 - mae: 2.2911
Epoch 11/20
303/303 [================

303